In [2]:
import sys,re,os
import pandas as pd
from ete4 import Tree

In [5]:
t=Tree(open('bin_ca_tetra_r220_archaea/gtdbtk.infer_RED_ranks.tree'),parser=1)

In [ ]:
print(t)

In [3]:
matches=[]
def find_mono(node,matches):
    children=node.get_children()
    if len(children)==2:
        a=list(map(lambda x: re.match('^FDZ', x),node.leaf_names()))
        b=list(filter(lambda x: x is not None, a))
        if len(a)==len(b) and len(a)>0: 
            matches.append(node)
        elif children[0].is_leaf and not children[1].is_leaf and re.match('^FDZ',children[0].name) is not None:
            matches.append(children[0])
            find_mono(children[1],matches)
        elif children[1].is_leaf and not children[0].is_leaf and re.match('^FDZ',children[1].name) is not None:
            matches.append(children[1])
            find_mono(children[0],matches)
        else:
            find_mono(children[0],matches)
            find_mono(children[1],matches)
    

In [6]:
find_mono(t,matches)

In [7]:
len(matches)

173

In [28]:
list(matches[1].ancestors(include_root=False))[0].name

'1.0|RED=0.895|genus'

In [15]:
matches[3].get_sisters()

[<Tree '1.0|RED=0.951|genus' at 0x7fe886e363b>]

In [112]:
def inferred_rank(node):
    rank=["phylum","class","order","family","genus"]
    if node.is_leaf:
        p=rank.index(re.sub(r"&.*","",list(node.ancestors(include_root=False))[0].name.split("|")[2]))
        return rank[p]
    n=rank.index(re.sub(r"&.*","",node.name.split("|")[2]))
    p=rank.index(re.sub(r"&.*","",list(node.ancestors(include_root=False))[0].name.split("|")[2]))
    if n==p or n==p+1: #if node's rank is equal to parent's rank or below 
        return rank[n]
    elif n>p+1:
        return rank[p+1]

def pull_mono_node_info(node):
    if node.is_leaf:
        # Leaf RED is set to 1, which may not be able to represent the monophylegetic branch
        # Use parent's RED and rank to represent leaf
        parent=list(node.ancestors(include_root=False))[0]
        rank=inferred_rank(node)
        node_name=parent.name.split('|')
        RED=re.search(r'RED=([\d\.]+)',node_name[1]).group(1)
        all_leaf=node.name
    else:
        all_leaf=",".join(list(node.leaf_names()))
        node_name=node.name.split('|')
        RED=re.search(r'RED=([\d\.]+)',node_name[1]).group(1)
        rank=inferred_rank(node)
    return pd.DataFrame({all_leaf:{'RED':RED,'rank':rank}})

In [9]:
arc_red=pd.concat(list(map(pull_mono_node_info,matches)),axis=1).T

In [88]:
arc_red.reset_index(names="Leaf_Names").to_csv('MEERv21_d95_gtdbr220_ar53_monophylogetic_RED.tsv',sep="\t",index=None)

In [75]:
list(filter(lambda x: x is not None, map(lambda x: re.search('FDZ070-GBeBe8-10.bin.193',x),arc_red.index.to_list())))

[<re.Match object; span=(46, 70), match='FDZ070-GBeBe8-10.bin.193'>]

In [10]:
arc_sgbt=Tree(open('bin_ca_tetra_user_arctree_r220/gtdbtk.decorated.tree'),parser=1)

In [29]:
'FDZ061-WW30-32.bin.394' in list(arc_sgbt.leaf_names())

True

In [70]:
tmp=[]

for node in matches:
    if node.is_leaf:
        #print(node.name)
        sgbt_node=arc_sgbt[node.name]
        #print(sgbt_node)
        tmp.append(sgbt_node)
    else:
        name_group=list(node.leaf_names())
        sgbt_node=arc_sgbt.common_ancestor(name_group)
        if len(sgbt_node)==len(node):
            tmp.append(sgbt_node)
        elif len(sgbt_node)>len(node):
            #tmp2.append(node)
            sgbt_node=arc_sgbt.common_ancestor(name_group)
            sgbt_o=list(map(lambda x: arc_sgbt[x], name_group))
            sgbt_e=list(sgbt_node.leaves())
            looking=list(set(sgbt_e).difference(set(sgbt_o)))
            print(looking)
            for i in looking:
                parent=list(i.ancestors())[0]
                if i.is_leaf:
                    if t[i.name] in matches:
                        #print(i.name)
                        if arc_sgbt[i.name] not in tmp:
                            tmp.append(arc_sgbt[i.name])
                        continue
                    elif not arc_red[arc_red.index.to_series().str.contains(i.name)].empty:
                        if parent is not sgbt_node and parent not in looking: looking.append(parent)
                        continue
                #print(i)
                sis=list(i.get_sisters())[0]
                #print(sis)
                if sis.is_leaf and sis.name in name_group:
                    tmp.append(sis)
                elif not sis.is_leaf:
                    if all(list(map(lambda x: x in name_group,list(sis.leaf_names())))):
                        #print(sis)
                        tmp.append(sis)
                
                if parent is not sgbt_node and parent not in looking: looking.append(parent)


[<Tree 'FDZ070-GBkY10-12.bin.165' at 0x7f2928c1451>]
[<Tree 'FDZ058-Be2-4.bin.360' at 0x7f2928bda50>]
[<Tree 'FDZ077-WuWW8-10.bin.175' at 0x7f2928bc8eb>, <Tree 'FDZ061-WW20-22.bin.69' at 0x7f2928bc8e4>, <Tree 'FDZ070-GBkY0-2.bin.307' at 0x7f2928bc8dd>]
[<Tree 'FDZ076-WRG2-4.bin.98' at 0x7f2928bcb53>, <Tree 'FDZ041-Be15-20.bin.806' at 0x7f2928bcb4c>]
[<Tree 'FDZ076-RGWu26-28.bin.341' at 0x7f2928bc835>]
[<Tree 'FDZ076-WWWu2-4.bin.343' at 0x7f2928bde81>, <Tree 'FDZ072-WRR4-6.bin.477' at 0x7f2928bdee3>, <Tree 'FDZ070-GBkYWu.bin.232' at 0x7f2928bdea4>, <Tree 'FDZ070-GBeBe8-10.bin.193' at 0x7f2928bde88>, <Tree 'FDZ071-RRG16-18.bin.388' at 0x7f2928bdeab>, <Tree 'FDZ081-YWG26-28.bin.99' at 0x7f2928bde6c>, <Tree 'FDZ060-GBkWNs.bin.684' at 0x7f2928bdeb2>, <Tree 'FDZ074-WuYW10-12.bin.5' at 0x7f2928bde73>, <Tree 'FDZ050-RBk28-30.bin.295' at 0x7f2928bded5>, <Tree 'FDZ062-GYBe10-12.bin.4' at 0x7f2928bde57>, <Tree 'FDZ066-ww10-12.bin.124' at 0x7f2928bde7a>, <Tree 'FDZ061-WW8-10.bin.53' at 0x7f2928bde

In [86]:
pd.Series(list(map(lambda x: x.name if x.is_leaf else ",".join(list(x.leaf_names())),tmp))).to_csv("MEERv21_ar53_monophylogetic.list",header=None,index=None,sep="\t")

## Testing Code

In [ ]:
tmp=[]
tmp2=[]
for node in matches:
    if node.is_leaf:
        #print(node.name)
        sgbt_node=arc_sgbt[node.name]
        #print(sgbt_node)
        tmp.append(sgbt_node)
    else:
        name_group=list(node.leaf_names())
        sgbt_node=arc_sgbt.common_ancestor(name_group)
        if len(sgbt_node)==len(node):
            tmp.append(sgbt_node)
        elif len(sgbt_node)>len(node):
            tmp2.append(node)

In [77]:
len(tmp)

184

In [14]:
len(tmp2)

7

In [15]:
len(matches)

173

In [72]:
tmp3=[]
c=0
node=tmp2[6]
name_group=list(node.leaf_names())

sgbt_node=arc_sgbt.common_ancestor(name_group)
sgbt_o=list(map(lambda x: arc_sgbt[x], name_group))
sgbt_e=list(sgbt_node.leaves())
looking=list(set(sgbt_e).difference(set(sgbt_o)))
print(looking)
for i in looking:
    parent=list(i.ancestors())[0]
    if i.is_leaf:
        if t[i.name] in matches:
            #print(i.name)
            if arc_sgbt[i.name] not in tmp:
                tmp3.append(arc_sgbt[i.name])
            continue
        elif not arc_red[arc_red.index.to_series().str.contains(i.name)].empty:
            if parent is not sgbt_node and parent not in looking: looking.append(parent)
            continue
    #print(i)
    sis=list(i.get_sisters())[0]
    #print(sis)
    if sis.is_leaf and sis.name in name_group:
        tmp3.append(sis)
    elif not sis.is_leaf:
        if all(list(map(lambda x: x in name_group,list(sis.leaf_names())))):
            #print(sis)
            tmp3.append(sis)
    c+=1
    if parent is not sgbt_node and parent not in looking: looking.append(parent)

[<Tree 'FDZ077-WuYW10-12.bin.330' at 0x7f2928c33d8>, <Tree 'FDZ070-RYBk14-16.bin.259' at 0x7f2928c33c3>, <Tree 'FDZ070-GBeBe12-14.bin.337' at 0x7f2928c33bc>, <Tree 'FDZ076-WWWu2-4.bin.420' at 0x7f2928c33df>]


In [73]:
print(node)
print(len(node))

─┬╴FDZ046-RBk0-5.bin.47
 ╰╴FDZ062-WuWR4-6.bin.144
2


In [74]:
print(sgbt_node)

 ╭╴FDZ062-WuWR4-6.bin.144
─┤ ╭─┬╴FDZ070-GBeBe12-14.bin.337
 ╰─┤ ╰╴FDZ070-RYBk14-16.bin.259
   ╰─┬╴FDZ046-RBk0-5.bin.47
     ╰─┬╴FDZ077-WuYW10-12.bin.330
       ╰╴FDZ076-WWWu2-4.bin.420


In [57]:
print(c)

3


In [88]:
arc_red[arc_red.index.to_series().str.contains('FDZ076-WRG2-4.bin.98')]

,RED,rank
"FDZ041-Be15-20.bin.806,FDZ076-WRG2-4.bin.98",0.856,genus


In [42]:
arc_sgbt['FDZ070-GBkY10-12.bin.165'] in tmp

False

In [75]:
tmp3

[<Tree 'FDZ046-RBk0-5.bin.47' at 0x7f2928c33d1>,
 <Tree 'FDZ062-WuWR4-6.bin.144' at 0x7f2928c33b5>]

In [69]:
print(tmp3[0])
print(len(tmp3[0]))
print(t.common_ancestor(list(tmp3[0].leaf_names())))

   ╭╴FDZ070-WuRG4-6.bin.199
 ╭─┤ ╭─┬╴FDZ058-Be16-18.bin.660
 │ ╰─┤ ╰╴FDZ070-RYBk22-24.bin.418
─┤   ╰─┬╴FDZ070-RYBk18-20.bin.64
 │     ╰─┬╴FDZ062-RRR0-2.bin.159
 │       ╰╴FDZ063-WW2-4.bin.39
 ╰─┬╴FDZ064-BaGW12-14.bin.73
   ╰─┬╴FDZ071-BuBu12-14.bin.75
     ╰─┬╴FDZ066-BawG2-4.bin.39
       ╰╴FDZ070-GBkY2-4.bin.348
10
 ╭─┬╴FDZ070-RYBk22-24.bin.418
 │ ╰─┬╴FDZ070-WuRG4-6.bin.199
 │   ╰─┬╴FDZ058-Be16-18.bin.660
─┤     ╰─┬╴FDZ070-RYBk18-20.bin.64
 │       ╰─┬╴FDZ062-RRR0-2.bin.159
 │         ╰╴FDZ063-WW2-4.bin.39
 ╰─┬╴FDZ064-BaGW12-14.bin.73
   ╰─┬╴FDZ071-BuBu12-14.bin.75
     ╰─┬╴FDZ066-BawG2-4.bin.39
       ╰╴FDZ070-GBkY2-4.bin.348


## Bacteria RED Tree

In [91]:
bact=Tree(open("bin_ca_tetra_r220_bacteria/gtdbtk.infer_RED_rankstree"),parser=1)

In [92]:
bac_matches=[]
find_mono(bact,bac_matches)

In [93]:
bac_RED=pd.concat(list(map(pull_mono_node_info,bac_matches)),axis=1).T

In [94]:
bac_RED.reset_index(names="Leaf_Names").to_csv("MEERv21_d95_gtdbr220_bac120_monophylogetic_RED.tsv",sep="\t",index=None)

In [95]:
bac_sgbt=Tree(open("bin_ca_tetra_user_bactree_r220/gtdbtk.decorated.tree",'rt'),parser=1)

In [ ]:
tmp=[]

for node in bac_matches:
    if node.is_leaf:
        #print(node.name)
        sgbt_node=bac_sgbt[node.name]
        #print(sgbt_node)
        tmp.append(sgbt_node)
    else:
        name_group=list(node.leaf_names())
        sgbt_node=bac_sgbt.common_ancestor(name_group)
        if len(sgbt_node)==len(node):
            tmp.append(sgbt_node)
        elif len(sgbt_node)>len(node):
            #tmp2.append(node)
            sgbt_node=bac_sgbt.common_ancestor(name_group)
            sgbt_o=list(map(lambda x: bac_sgbt[x], name_group))
            sgbt_e=list(sgbt_node.leaves())
            looking=list(set(sgbt_e).difference(set(sgbt_o)))
            #print(looking)
            for i in looking:
                parent=list(i.ancestors())[0]
                if i.is_leaf:
                    if bact[i.name] in bac_matches:
                        #print(i.name)
                        if bac_sgbt[i.name] not in tmp:
                            tmp.append(bac_sgbt[i.name])
                        continue
                    elif not bac_RED[bac_RED.index.to_series().str.contains(i.name)].empty:
                        if parent is not sgbt_node and parent not in looking: looking.append(parent)
                        continue
                #print(i)
                sis=list(i.get_sisters())[0]
                #print(sis)
                if sis.is_leaf and sis.name in name_group:
                    tmp.append(sis)
                elif not sis.is_leaf:
                    if all(list(map(lambda x: x in name_group,list(sis.leaf_names())))):
                        #print(sis)
                        tmp.append(sis)
                
                if parent is not sgbt_node and parent not in looking: looking.append(parent)


In [98]:
len(tmp)

2188

In [103]:
bac_RED.index.to_series().str.contains(",").to_frame(name="bool").groupby('bool').size()

bool
False     887
True     1048
dtype: int64

In [106]:
arc_red.index.to_series().str.contains(",").to_frame(name="bool").groupby('bool').size()

bool
False     72
True     101
dtype: int64

In [104]:
bac_RED.shape

(1935, 2)

In [105]:
arc_red.shape

(173, 2)

In [99]:
pd.Series(list(map(lambda x: x.name if x.is_leaf else ",".join(list(x.leaf_names())),tmp))).to_csv("./MEERv21_bac120_monophylogetic.list",header=None,index=None,sep="\t")